In [1]:
import re
import pandas as pd
import json
import requests
import FinanceDataReader as fdr
import numpy as np
from bs4 import BeautifulSoup
from tqdm import tqdm
from datetime import datetime

In [4]:
#특정 날짜 23년 2월 2일 크롤링
url = 'http://www.fantastock.co.kr/bbs/board.php?bo_table=free_list23&wr_id=2190#section1'
response = requests.get(
    url,
    headers={
        'User-Agent': 'Mozilla 5.0'
    }
    )
bs = BeautifulSoup(response.text, 'lxml')    

In [167]:
#특정 날짜 23년 2월 2일 크롤링
df = pd.DataFrame(columns=['comment'])
for i in range(len(bs.select('.cmt_contents'))):
    comment = bs.select('.cmt_contents')[i].text
    df.loc[i] = [comment]

In [4]:
#게시판에서 각 게시물 주소얻기 
url = 'http://www.fantastock.co.kr/bbs/board.php?bo_table=free_list23'
response = requests.get(
    url,
    headers={
        'User-Agent': 'Mozilla 5.0'
    }
    )
bs = BeautifulSoup(response.text, 'lxml') 

In [200]:
#1페이지 링크 얻기
for i in range(len(bs.select('table>tr'))):
    date = bs.select('table>tr')[i].select('dd')[1].text
    url_link = bs.select('table>tr')[i].select('a')[0].get('href')

In [5]:
#해당 년도 모든 페이지에 날짜 url 얻기
#마지막 페이지 찾는 작업 필요 
for page_num in range(1,4):
    url = f'http://www.fantastock.co.kr/bbs/board.php?bo_table=free_list22&page={page_num}'
    response = requests.get(
        url,
        headers={
        'User-Agent': 'Mozilla 5.0'
        }
        )
    bs = BeautifulSoup(response.text, 'lxml') 

    for i in range(len(bs.select('table>tr'))):
        date = bs.select('table>tr')[i].select('dd')[1].text
        url_link = bs.select('table>tr')[i].select('a')[0].get('href')

In [8]:
#함수화
def get_info_from_url(url):
    response = requests.get(
        url,
        headers={
        'User-Agent': 'Mozilla 5.0'
        }
    )
    bs = BeautifulSoup(response.text, 'lxml') 
    return bs

In [9]:
#모든 년도 모든 페이지에 날짜 url 얻기
#마지막 페이지 찾는 작업 필요 
df = pd.DataFrame()
for year in tqdm(range(19,24)):
    for page_num in tqdm(range(1,4)):
        url = f'http://www.fantastock.co.kr/bbs/board.php?bo_table=free_list{year}&page={page_num}'
        bs = get_info_from_url(url)  
        info_from_bs = bs.select('table>tr')
        for bs in info_from_bs:
            try :
                date = bs.select('dd')[1].text
                url_link = bs.select('a')[0].get('href')
                bs = get_info_from_url(url_link)
                data_from_url = bs.select('.cmt_contents')

                for comments in data_from_url:
                    comment = comments.text

                    temp = pd.DataFrame({
                        'date': date,
                        'url': url_link,
                        'comment': comment
                        }, index=[0])

                    df = pd.concat([df, temp], axis=0)
            except IndexError:
                print(1)

100%|██████████| 5/5 [02:36<00:00, 31.31s/it]

1
1


In [41]:
#파일로 저장
df.to_csv('data.csv', index=False, encoding='utf-8-sig')


In [59]:
#파일 불러오기
df = pd.read_csv('./data.csv')

In [60]:
#금일 추천주 필터링
df = df[df['comment'].str.contains('매수', na =False)]
df = df[df['comment'].str.contains('현재시간', na =False)]
df = df[df['comment'].str.contains('▶', na =False)]
df['comment'] = df.comment.apply(lambda x : re.sub('\\n','',x))

In [61]:
#date 를 datetime 형식으로 변환 1
df['date'] = df.date.apply(lambda x : x.replace(".",""))
df['date']= df['date'].astype('str')
df['date'] = df.date.apply(lambda x : datetime.strptime(x,'%Y%m%d'))
df = df.reset_index(drop=True)

In [63]:
#date 를 datetime 형식으로 변환 2
#df2 = df 
#df2['date'] = pd.to_datetime(df2['date'])

In [64]:
# # 예외찾기
# df2 = df['slicing_comment'].apply(lambda x : x == [] )
# df2[df2]

In [65]:
#예외처리
df.comment[54] = df.comment[54].replace('\xa0',' ')

C:\Users\admin\AppData\Local\Temp\ipykernel_16464\3069763651.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.comment[54] = df.comment[54].replace('\xa0',' ')


In [66]:
#종목 슬라이스
df['slicing_comment'] = df.comment.apply(lambda x : re.findall('\d+분 ([가-힣A-z]+)',x)[0])

In [67]:
#종목명 추출
recommended_stock = df.filter(items=['date','slicing_comment'])
recommended_stock = recommended_stock.rename(columns={'slicing_comment':'stock_name'})

In [68]:
#데이터프레임 한번에 보기
#pd.set_option('display.max_rows', None)
#다시 돌리기
#pd.options.display.max_rows = 100

In [69]:
#대소변환
recommended_stock['stock_name'] = recommended_stock.stock_name.str.upper()

In [70]:
#주가 기본정보 불러오기
Kospi_df = pd.read_csv('./kospi_code.csv')
Kosdaq_df = pd.read_csv('./kosdaq_code.csv')
kospi = Kospi_df.filter(items= ['단축코드','한글명'])
kosdaq = Kosdaq_df.filter(items= ['단축코드','한글종목명'])

In [71]:
#주가 종목코드 불러오기
kospi = kospi.rename(columns={'단축코드':'stock_code','한글명' : 'stock_name'})
kosdaq = kosdaq.rename(columns={'단축코드':'stock_code','한글종목명' : 'stock_name'})
kospi_kosdaq = pd.concat([kospi,kosdaq])

In [72]:
#종목 코드 합치기 1트
stock = pd.merge(recommended_stock, kospi_kosdaq, how = 'left', on  = 'stock_name')

In [73]:
#누락된NaN 버리기
stock1 = stock.dropna()

In [74]:
#누락된NaN 값들중 주울거 있는지 확인하기 
stock2 = stock.drop(stock1.index,axis=0)
stock2 = stock2.drop(['stock_code'],axis=1)

In [75]:
#누락된 종목코드 찾기
# exception = stock[stock.stock_code.isnull()]
exception = stock.drop(stock1.index,axis=0)

In [36]:
#파일로 저장
#exception.to_csv('exception.csv', index=False, encoding='utf-8-sig')

In [76]:
#파일불러오기
exception = pd.read_csv('./exception.csv')

In [77]:
#exception 정제
exception = exception.dropna()
exception['stock_code'] = exception.stock_code.apply(lambda x : str(int(x)).zfill(6))
exception = exception.filter(items = ['stock_name','stock_code'])

In [78]:
#종목 코드 합치기 2트
stock2 = pd.merge(stock2, exception, how = 'left', on  = 'stock_name')
stock2 = stock2.dropna()

In [79]:
#최종 stock 가져오기 stock = stock1+stock2
stock = pd.concat([stock1,stock2])
stock = stock.sort_values(by=['date'], ascending=[True])
stock = stock.reset_index(drop= True)

In [40]:
#파일로 저장
# stock.to_csv('recommend_stock.csv', index=False, encoding='utf-8-sig')
#파일 불러오기
stock = pd.read_csv('./recommend_stock.csv')

In [41]:
stock

,date,stock_name,stock_code
0,2019-10-29,알리코제약,260660
1,2019-10-29,KPX생명과학,114450
2,2019-10-29,제일바이오,052670
3,2019-10-30,아스타,246720
4,2019-10-30,SM,063440
...,...,...,...
977,2023-02-01,오픈베이스,049480
978,2023-02-01,일성건설,013360
979,2023-02-02,선익시스템,171090
980,2023-02-02,누리플렉스,040160


In [4]:
stock['strdate'] = stock.date.apply(lambda x: re.sub("-","",x))

In [2]:
#한국투자 API 보안 인증키 발급 
APP_KEY = "PSDbkNNrIX3j60cW2MEFnBGZDZ1BzpgDxvC0"
APP_SECRET = "QfaZRX24K15ojO4yjG5o5dM+SUvJT4PYiK+tKKzgikyhmfrnTXoxOwn9rCf2hzXdtOhh4v3/O4bSFfQgS+u6ojdb6KgWuyub4GRraoWYaJTJyHSRqWUPab2GiCvrI/aIjx5S925MohrETq0k/eopFHgMInG0lmZdAh5TaegaZ1NDpNvPW8Y="
URL_BASE = "https://openapi.koreainvestment.com:9443"
PATH = "oauth2/tokenP"
URL = f"{URL_BASE}/{PATH}"

headers = {"content-type":"application/json"}
body = {"grant_type":"client_credentials",
        "appkey":APP_KEY, 
        "appsecret":APP_SECRET}

res = requests.post(URL, headers=headers, data=json.dumps(body))
ACCESS_TOKEN = res.json()["access_token"]

In [22]:
#현재가 가져오기 
def get_info_from_kisapi(stock_code,date):
    URL = 'https://openapi.koreainvestment.com:9443/uapi/domestic-stock/v1/quotations/inquire-daily-itemchartprice'

    headers = {"Content-Type":"application/json", 
            "authorization": f"Bearer {ACCESS_TOKEN}",
            "appKey":APP_KEY,
            "appSecret":APP_SECRET,
            "tr_id":"FHKST03010100"
            }

    params = {
        "FID_COND_MRKT_DIV_CODE":"J",
        "FID_INPUT_ISCD":stock_code,
        "FID_INPUT_DATE_1" : date,
        "FID_INPUT_DATE_2" : date,
        "FID_PERIOD_DIV_CODE" : "D",
        "FID_ORG_ADJ_PRC" : "1"
    }

    res = requests.get(URL, headers=headers, params=params)
    stck_oprc = res.json().get('output2')[0].get('stck_oprc')
    
    return stck_oprc

In [15]:
#개장일 계산기
def checking_tr_day(date):
    URL = 'https://openapi.koreainvestment.com:9443/uapi/domestic-stock/v1/quotations/inquire-daily-itemchartprice'

    headers = {"Content-Type":"application/json", 
        "authorization": f"Bearer {ACCESS_TOKEN}",
        "appKey":APP_KEY,
        "appSecret":APP_SECRET,
        "tr_id":"CTCA0903R"
        }

    params = {
    "BASS_DT": date,
    "CTX_AREA_NK":"",
    "CTX_AREA_FK":""
    }

    res = requests.get(URL, headers=headers, params=params)
    return res


In [37]:
date = pd.date_range('20180101', '20230217')
date = date.strftime("%Y%m%d")

In [39]:
df_krx = fdr.StockListing('KRX')

In [42]:
stock['date'] = stock.date.apply(lambda x : re.sub("-",".",x))
stock['date']= stock['date'].astype('str')
stock['date'] = stock.date.apply(lambda x : datetime.strptime(x,'%Y.%m.%d'))


In [44]:
stock

,date,stock_name,stock_code
0,2019-10-29,알리코제약,260660
1,2019-10-29,KPX생명과학,114450
2,2019-10-29,제일바이오,052670
3,2019-10-30,아스타,246720
4,2019-10-30,SM,063440
...,...,...,...
977,2023-02-01,오픈베이스,049480
978,2023-02-01,일성건설,013360
979,2023-02-02,선익시스템,171090
980,2023-02-02,누리플렉스,040160


In [61]:
date = pd.date_range(end = stock.date[0],periods=10)
date = date.strftime("%Y.%m.%d")
date2 = date[0]
date3= date[-1]

In [63]:
date2

'2019.10.20'

In [64]:
stock_price = fdr.DataReader(stock.stock_code[0], date2, date3)


In [83]:
stock_price.iloc[-5:-1,:].filter(items = ['Close','Volume'])

,Close,Volume
Date,,
2019-10-23,8376,1510701
2019-10-24,7958,383465
2019-10-25,7606,190690
2019-10-28,7830,2999561


In [87]:
stock_price.iloc[-1:,:].filter(items = ['Open','Volume'])

,Open,Volume
Date,,
2019-10-29,8599,7971137
